# [Langsmith](https://python.langchain.com/v0.1/docs/langsmith/walkthrough/) + [Langgraph](https://langchain-ai.github.io/langgraph/#example) + [langchain_neospace](https://github.com/neospace-ai/langchain-neospace/releases)

## Setup

### Installation

In [1]:
%pip install -U langgraph langsmith
%pip install langchain-openai
%pip install python-dotenv
%pip install --upgrade --quiet  langchain langchainhub
%pip install --upgrade --quiet  tiktoken pandas duckduckgo-search
%pip install langchain_community
%pip install https://github.com/neospace-ai/langchain-neospace/releases/download/v1.0.0-TuSpX3J7/langchain_neospace-0.1.18-py3-none-any.whl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached https://github.com/neospace-ai/neospace-python/releases/download/v1.0.0-unXEA9WF/neospace-1.37.0-py3-none-any.whl (339 kB)
Note: you may need to restart the kernel to use updated packages.


### Env

In [2]:
from dotenv import load_dotenv, find_dotenv
import os
dotenv = find_dotenv()
print(f"Found .env file at {dotenv}")
load_dotenv(dotenv_path=dotenv, override=True)

from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"


Found .env file at /home/guilherme-alves-carvalho/Documents/repos/dsc-langgraph/.env


### Helpers

In [3]:
import json
import datetime
from langchain_core.messages import BaseMessage, HumanMessage

class LLMLogger:
    def __init__(self, wrapped_class):
        self.wrapped_class = wrapped_class
        run_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        os.makedirs(f"logs/{run_date}", exist_ok=True)

        self.req_res_f = f"logs/{run_date}/exec.json"

        self.req_res = []

    def __del__(self):
        self.write_to_file()

    def write_to_file(self):
        with open(self.req_res_f, "w+") as f:
            json.dump(self.req_res, f, indent=4, ensure_ascii=False)
        

    def __getattr__(self, attr):
        original_func = getattr(self.wrapped_class, attr)

        def wrapper(*args, **kwargs):

            result = original_func(*args, **kwargs)

            self.req_res.append({"request": kwargs, "response": result.model_dump()})
            return result

        return wrapper
    
def show_message(message: BaseMessage):
    if message.type == "assistant" and message.function_call:
        print(f"{message.type} function call => {message.function_call}")

    print(f"{message.type} => {message.content}")
    


## Workflow

### Build

In [11]:

from typing import Annotated, Literal

from langchain_core.messages import SystemMessage

from langgraph.checkpoint import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

import os

from langchain import hub
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_neospace import ChatNeoSpace

# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
# https://smith.langchain.com/hub/wfh/langsmith-agent-prompt?organizationId=3e9f24f6-5a49-5652-a085-d776844de3bf
# Fetches the latest version of this prompt
prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")

@tool
def executar_pix(query: str) :
    """Executar pix"""
    return {"status": 500}


# Define the tools for the agent to use
tools = [
    DuckDuckGoSearchResults(
        name="duck_duck_go"
        # type="DEFAULT"      
    ),  # General internet search using DuckDuckGo
    # executar_pix
]
# implement an user tool

tool_node = ToolNode(tools)

extra_body = {"session_id": "ERHJWKRHWQE"}

model = ChatNeoSpace(
    model="/cortex/checkpoints/cortex-7b-instruct-0.0.1",
    temperature=0,
    extra_body=extra_body,
)

model.client = LLMLogger(model.client)

model = model.bind_tools(tools, extra_body=extra_body)

class State(MessagesState):
    extra_body: Annotated[dict, "The configuration for the workflow"] # validation needs

# Define the function that determines whether to continue or not
def should_continue(state: State) -> Literal["tools", "agent", END]:
    messages = state['messages']
    last_message = messages[-1]

    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"

    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: State):
    messages = state['messages']
  
    response = model.bind(extra_body=state["extra_body"]).invoke(messages)
    # We return a list, because this will get added to the existing list

    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(State)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

### Run

In [12]:
from langchain_core.tracers.context import tracing_v2_enabled

from langsmith import Client
langsmith_client = Client(api_url=os.getenv("MERCURY_ENDPOINT"))
with tracing_v2_enabled(client=langsmith_client):
    extra_body["customer"] = {
        "id": "45646",
        "name": "Felipe Silva",
        "gender": "male"
    }

    state = {"messages": [SystemMessage(content="Você é um agente especialista financeiro, mostre todos os seus cálculos realizados em sua resposta")], "extra_body": extra_body}
    show_message(state["messages"][-1])

    question = HumanMessage(content=input())
    state["messages"] += [question]

    show_message(state["messages"][-1])


    while state["messages"][-1].content != "q":
        state = app.invoke(
            state,
            config={"thread_id": 42, "workflow": workflow},
        )

        show_message(state["messages"][-1])

        question = HumanMessage(content=input())
        state["messages"] += [question]
        show_message(state["messages"][-1])

        
    model.client.write_to_file()


system => Você é um agente especialista financeiro, mostre todos os seus cálculos realizados em sua resposta
human => Oi
ai =>  [{"name": "duck_duck_go", "arguments": {"query": "financial calculations excel formulas"}}]

Aqui estão alguns dos melhores recursos de cálculos financeiros disponíveis no Excel:

1. Função SUM: Somar valores em uma célula ou um conjunto de células.
   Fórmula: `=SUM(A1:A10)`

2. Função COUNT: Contar a quantidade de valores numa célula ou um conjunto de células.
   Fórmula: `=COUNT(B1:B10)`

3. Função AVERAGE: Calcular a média dos valores numa célula ou um conjunto de células.
   Fórmula: `=AVERAGE(C1:C10)`

4. Função MAX e MIN: Encontrar o maior e o menor valor numa célula ou um conjunto de células.
   Fórmula: `=MAX(D1:D10)` ou `=MIN(D1:D10)`

5. Função IF: Executar uma ação condicional.
   Fórmula: `=IF(E1>100,"Acima do valor", "Abaixo do valor")`

6. Função VLOOKUP: Localizar um valor em uma coluna e retornar um valor em uma célula relacionada.
   Fórmula: